# coding time Prediction

In [1]:
import pandas as pd
import csv
from datetime import datetime

# 1. Prepare training dataset

## 1.1 import training data

In [2]:
df = pd.read_csv('./commitLogDataFromOpenSource.csv', sep=';')
df = df[pd.notnull(df['user'])]
print(df.head(100))

           hash                user                            time  \
0   631996802a5         Logan Ramos  Mon Jun 28 11:54:00 2021 -0400   
1   31944f3ca92     Brigit Murtaugh  Mon Jun 28 08:48:17 2021 -0700   
2   5b29adfa24d         Logan Ramos  Mon Jun 28 11:25:15 2021 -0400   
3   d1bb28cb08f        Raymond Zhao  Mon Jun 28 08:05:04 2021 -0700   
4   e4bf568ed64         Logan Ramos  Mon Jun 28 10:28:43 2021 -0400   
..          ...                 ...                             ...   
95  6c3defb810b     Johannes Rieken  Fri Jun 25 15:44:27 2021 +0200   
96  f9f0798c530  Henning Dieterichs  Fri Jun 25 15:22:19 2021 +0200   
97  72e47d9dfec     Johannes Rieken  Fri Jun 25 15:07:46 2021 +0200   
98  ada9e8c66c0         Daniel Imms  Fri Jun 25 06:07:15 2021 -0700   
99  48d81c87547           Alex Ross  Fri Jun 25 14:43:03 2021 +0200   

                                               commit  
0   Don't show configure default if no file extension  
1                                  

## 1.2 store all usernames and times of commit in dictionary
* calculate one code line change taking how much time for each person


In [3]:
# put all users to dictionary users.
users = {}

for user in df['user']:
    if user not in users:
        users[user] = 1
    else:
        users[user] += 1
        
#if a user only appears in csv file once, we need to remove it from dictionary
all_users_dict = {}
for user, v in users.items():
    if v != 1:
        all_users_dict[user] = [v]
print(all_users_dict)


{'Logan Ramos': [176], 'Brigit Murtaugh': [10], 'Raymond Zhao': [101], 'Alex Ross': [1431], 'Daniel Imms': [3765], 'João Moreno': [1479], 'Andre Weinand': [705], 'Benjamin Pasero': [6762], 'Sandeep Somavarapu': [4805], 'Rob Lourens': [2948], 'Henning Dieterichs': [147], 'Jackson Kearl': [994], 'Johannes Rieken': [6418], 'isidor': [3589], 'Ladislau Szomoru': [261], 'Alexandru Dima': [904], 'rebornix': [2242], 'Alex Dima': [3012], 'Joyce Er': [7], 'SteVen Batten': [1153], 'meganrogge': [316], 'Connor Peet': [568], 'Megan Rogge': [173], 'deepak1556': [102], 'dependabot[bot]': [39], 'Isidor Nikolic': [321], 'Suzy Mueller': [6], 'Martin Aeschlimann': [2491], 'Miguel Solorio': [963], 'Rick Martin': [2], 'Jean Pierre': [52], 'Robo': [58], 'Dirk Baeumer': [425], 'Christof Marti': [767], 'Quentin Chateau': [4], 'Dirk Bäumer': [38], 'Kai Maetzel': [15], 'Matt Bierner': [5576], 'unknown': [8], 'Barbara Valdez': [3], 'Peng Lyu': [912], 'Greg Van Liew': [50], 'jeanp413': [123], 'gregvanl': [4], '大表

## 1.3 For each user in dictionary summe up lines of code changed

### 1.3.1 Get commit summary for each user
Accroding to the gaven username via command line get commit summary of the user.
Summary includes: 
* %ad: author date
* %s: comment mesage
* numstat: number of inserted and deleted lines changes

In [4]:
def get_user_commit_summary(username):
    # username = 'Logan Ramos'
    git_command = 'cd /Users/zhiyiniu/Repositories/vscode && git log  --date=local --all --pretty="%ad;;%s" --author="{}" --numstat'
    result = !{git_command.format(username)}
    # print(result)
    return result

#### result format: 
each user commit summary is composed of three parts: 
* commit time ;; commit message (two simicole points are splitor between time and comment)
* '' empty line
* number of changes in a file: starting with 1\t1\t+path
    > first number before \t is number of inserted lines
    > second number before \t is number of deleted lines

e.g.:
["Mon Jun 28 17:54:00 2021;;Don't show configure default if no file extension", '','1\t1\tsrc/vs/workbench/...']

### 1.3.2 Summe up lines of code changed in summary
take number of inserted lines and deleted lines as lines of code changed 
* Sometimes one commit includes more than one file --> "Every four commas is one commit" is wrong
* So we need to identify the begining of each string in array
  + remove all empty string
  + If it begins 'Sun', 'Mon', 'Tue', 'Wed', 'Thu' ,'Fri','Sat' --> Start a new commit
  + else is number of changes 

In [5]:
def summe_up_lines_of_chenged_code(summary):
    sum_of_code_changes = 0 
    time_list = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu' ,'Fri','Sat']
    # remove all empty element in list
    summary = list(filter(None, summary))
    
    commit_time_with_num_changes_and_comment = {}
    i = 0
    current_commit_time = ''
    new_result = []
    while i < len(summary)-1:
        if summary[i].split(' ')[0] in time_list:
            if i+1 < len(summary) and summary[i+1].split(' ')[0] not in time_list:
                current_commit_time = summary[i].split(';;')[0]
                commit_message = summary[i].split(';;')[1]
                commit_time_with_num_changes_and_comment[current_commit_time] = {}
                commit_time_with_num_changes_and_comment[current_commit_time]['num_lines_changed'] = 0
                commit_time_with_num_changes_and_comment[current_commit_time]['commit_messages'] = commit_message
        else:
            changes = summary[i].split('\t')
            # chnages_num = commit_time_with_num_changes_and_comment[current_commit_time]['num_lines_changed']
            # print(str(changes))
            try:
                commit_time_with_num_changes_and_comment[current_commit_time]['num_lines_changed'] += (int(changes[0]) + int(changes[1]))
            except:
                pass
        i = i + 1
    for key in commit_time_with_num_changes_and_comment:
        if commit_time_with_num_changes_and_comment[key]['num_lines_changed'] == 0:
            return {}
    return commit_time_with_num_changes_and_comment

## 1.4 Classification commit message

### 1.4.1 Get model variable in classification model file 

In [6]:
%store -r logregclf
logregclf

%store -r countvec
countvec

%store -r target_names
target_names

array(['backend', 'cloud', 'data-analysis', 'frontend', 'os'],
      dtype=object)

### 1.4.2 Test if stored variables work

In [7]:
input_data = []

test_sentence1 = "How can I configure DNS for my local Cloud Server?"
test_sentence2 = "No permission to open MySQL audit log."
input_data.append(test_sentence1)
input_data.append(test_sentence2)

sentences_categories = logregclf.predict(countvec.transform(input_data))
for data, category in zip(input_data, sentences_categories):
    print(f'The Jira Ticket "{data}" belongs to the category "{target_names[category]}".')

The Jira Ticket "How can I configure DNS for my local Cloud Server?" belongs to the category "cloud".
The Jira Ticket "No permission to open MySQL audit log." belongs to the category "data-analysis".


### import sentiment analysis model

In [8]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('vader_lexicon')

def get_sentiment_score(sentences):
    scores = []
    for sentence in sentences:
        sentence_score = SentimentIntensityAnalyzer().polarity_scores(sentence)["compound"]
        scores.append(sentence_score)
    # print(scores)
    return scores

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/zhiyiniu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### 1.4.3 Calculate end score 

In [15]:
def calculate_end_score(dict_):
    end_score = 0
    sum_lines = 0 
   
    for key in dict_:
        lines = dict_[key]['num_lines_changed']
        sum_lines += lines
        sentiment_score = dict_[key]['sentiment_score']
        end_score += lines*sentiment_score
    end_score = round(end_score/sum_lines,3)
    return end_score

### 1.4.4 Using Classification Model to classify commit message 

In [16]:
def classify_commit_messages(summary_dict):
    dict_ = {}
    # add each target_names as key in dict
    for name in target_names:
        if name not in dict_:
            dict_[name] = {}
            dict_[name]['num_lines_changed'] = 0
            # index 0 is sentiment score and index 1 is count
            dict_[name]['sentiment_score'] = [0, 0]
            
    # store commit messages in a list
    commit_messages = []
    num_lines_changed = []
    
    for key in summary_dict:
        message = summary_dict[key]['commit_messages']
        if message != '':
            commit_messages.append(message)
            num_lines_changed.append(summary_dict[key]['num_lines_changed'])
    
    sentiment_scores = get_sentiment_score(commit_messages)

    sentences_categories = logregclf.predict(countvec.transform(commit_messages))
    for data, category, num, score in zip(commit_messages, sentences_categories, num_lines_changed, sentiment_scores):
        category_name = target_names[category]
        dict_[category_name]['num_lines_changed'] += num
        dict_[category_name]['sentiment_score'][0] += score
        dict_[category_name]['sentiment_score'][1] += 1
        #print(f'The Git commit message "{data}" belongs to the category "{category_name}".')
        #print(dict_)
    #print('==========================')
    for key in dict_:
        count = dict_[key]['sentiment_score'][1]
        if count == 0:
            count = 1
        dict_[key]['sentiment_score'] = round((dict_[key]['sentiment_score'][0])/count,3)
    
    dict_['end_score'] = calculate_end_score(dict_)
    
    # print(dict_)
    return dict_

In [17]:
# classify_commit_messages(summary_dict)

# 2. For each author  

In [18]:
def iterator_all_users_and_get_result():
    result = {}
    for user in all_users_dict:
        # print('==========================')
        # print('User:' + user)
        if all_users_dict[user][0] > 50:
            result[user] = {}
            summary = get_user_commit_summary(user)
            if summary != []:
                summary_dict = summe_up_lines_of_chenged_code(summary)
                if summary_dict != {}:
                    result[user] = classify_commit_messages(summary_dict)
    return result

In [ ]:
result = iterator_all_users_and_get_result()

# 3. Get JSON output Result

In [ ]:
import json

def get_user_data_in_json(result):        
    data_in_json = json.dumps(result, indent=4)
    # print(data_in_json)
    return data_in_json

output_dict = {}
for key in result:
    if result[key] != {}:
        output_dict[key] = result[key]
            
output_in_json = get_user_data_in_json(output_dict)
print(output_in_json)